In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Function to extract keywords from a text
def extract_keywords(texts, n_keywords=5):
    vectorizer = CountVectorizer(max_features=1000, stop_words='english')
    X = vectorizer.fit_transform(texts)
    word_sum = np.array(X.sum(axis=0)).flatten()
    words = np.array(vectorizer.get_feature_names_out())
    sorted_idx = np.argsort(word_sum)[::-1]
    return [", ".join(words[sorted_idx][:n_keywords])]

# Load the Excel file

df = pd.read_excel('QvsK.xlsx')

# Apply the function to extract keywords for each question
df['Keywords'] = df['TITLE_E'].apply(lambda x: extract_keywords([x]))

# Show the updated DataFrame
print(df.head())


  QUESTION                                            TITLE_E  \
0      Q01  I have the tools, technology and equipment I n...   
1      Q02  The material and tools provided for my work, i...   
2      Q03  My physical environment (e.g., office, workspa...   
3      Q04            I get the training I need to do my job.   
4      Q05  I have the information, training and equipment...   

                                            Keywords  
0          [tools, technology, need, job, equipment]  
1        [tools, work, software, provided, official]  
2  [workspace, suitable, requirements, physical, ...  
3                              [training, need, job]  
4        [work, training, safety, need, information]  


In [ ]:
df.to_excel('2nd_output_keys.xlsx')

In [ ]:
import pandas as pd
import spacy
from gensim import corpora, models

# Data
data = [['tools  technology need job equipment'],
        ['tools work software provided official'],
        ['workspace suitable requirements physical office'],
        ['training need job'],
        ['work training safety need information'],
        ['work support personal life balance']]

df = pd.DataFrame(data, columns=['Keyword'])
# Load spaCy English model (download if not installed)
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
  # Lowercase, remove stopwords, and lemmatize
  doc = nlp(text.lower())
  tokens = [token.lemma_ for token in doc if token.is_alpha]  # Keep only alphabetic tokens
  return " ".join(tokens)



# Preprocess each row
df['Processed_Text'] = df.apply(lambda row: " ".join(row.iloc[:5]), axis=1)
df['Processed_Text'] = df['Processed_Text'].apply(preprocess_text)
# Create dictionary (mapping words to unique IDs)
dictionary = corpora.Dictionary(df['Processed_Text'])

# Create DTM
dtm = [dictionary.doc2bow(text) for text in df['Processed_Text']]
# Train LDA model (adjust num_topics as needed)
lda_model = models.LdaModel(dtm, id2word=dictionary, num_topics=3)

# Get topics and top words for each topic
topics = lda_model.show_topics(formatted=False)

# Assign topic to each row based on the dominant topic
df['Topic'] = df['Processed_Text'].apply(lambda text: lda_model[dictionary.doc2bow(text)][0][1]).astype(str)  # Extract dominant topic ID and convert to string

print(df)

NameError: name 'tokens' is not defined